* 데이터출처 : 서울 열린데이터광장

* 서울시 버스노선별 정류장별 시간대별 승하차 인원 정보 : http://data.seoul.go.kr/dataList/OA-12913/S/1/datasetView.do
* 서울시 지하철 호선별 역별 시간대별 승하차 인원 정보 :http://data.seoul.go.kr/dataList/OA-12252/S/1/datasetView.do

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def get_font_family():
    """
    시스템 환경에 따른 기본 폰트명을 반환하는 함수
    """
    import platform
    system_name = platform.system()

    if system_name == "Darwin" :
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        # Linux(colab)
        !apt-get install fonts-nanum -qq  > /dev/null
        !fc-cache -fv

        import matplotlib as mpl
        mpl.font_manager._rebuild()
        findfont = mpl.font_manager.fontManager.findfont
        mpl.font_manager.findfont = findfont
        mpl.backends.backend_agg.findfont = findfont
        
        font_family = "NanumBarunGothic"
    return font_family

plt.rc("font", family=get_font_family())
plt.rc("axes", unicode_minus=False)

## 데이터 로드

In [3]:
raw = pd.read_csv("data/서울시 지하철 호선별 역별 시간대별 승하차 인원 정보.csv", encoding="cp949")
raw.shape

(45945, 52)

In [4]:
raw.head()

,사용월,호선명,지하철역,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,...,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원,작업일자
0,202107,1호선,동대문,665,13,13154,2112,8755,6552,13558,...,7234,7,1394,0,0,0,1,0,0,20210803
1,202107,1호선,동묘앞,92,0,3197,1077,3575,4910,5452,...,1955,3,1039,0,0,0,0,0,0,20210803
2,202107,1호선,서울역,654,6,8652,6342,11542,33836,34033,...,5824,29,551,0,0,0,0,0,0,20210803
3,202107,1호선,시청,13,0,1803,4616,2797,19665,6210,...,988,6,84,0,0,0,0,0,0,20210803
4,202107,1호선,신설동,295,14,7940,3120,7834,10994,15927,...,4104,21,437,1,0,0,0,0,0,20210803


In [5]:
raw.tail()

,사용월,호선명,지하철역,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,...,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원,작업일자
45940,201501,중앙선,운길산,0,0,177,56,642,403,1292,...,633,3,222,0,0,0,0,0,0,20151223
45941,201501,중앙선,원덕,1,0,292,4,415,46,537,...,383,3,157,0,0,0,0,0,0,20151223
45942,201501,중앙선,중랑,14,1,4929,424,8539,2084,22160,...,6501,86,1617,0,0,0,0,0,0,20151223
45943,201501,중앙선,팔당,0,0,161,50,368,149,1227,...,336,8,85,0,0,0,0,0,0,20151223
45944,201501,중앙선,회기,436,3,10929,3067,23317,10430,65043,...,35702,767,9314,0,0,0,0,0,0,20151223


In [6]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45945 entries, 0 to 45944
Data columns (total 52 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   사용월           45945 non-null  int64 
 1   호선명           45945 non-null  object
 2   지하철역          45945 non-null  object
 3   04시-05시 승차인원  45945 non-null  int64 
 4   04시-05시 하차인원  45945 non-null  int64 
 5   05시-06시 승차인원  45945 non-null  int64 
 6   05시-06시 하차인원  45945 non-null  int64 
 7   06시-07시 승차인원  45945 non-null  int64 
 8   06시-07시 하차인원  45945 non-null  int64 
 9   07시-08시 승차인원  45945 non-null  int64 
 10  07시-08시 하차인원  45945 non-null  int64 
 11  08시-09시 승차인원  45945 non-null  int64 
 12  08시-09시 하차인원  45945 non-null  int64 
 13  09시-10시 승차인원  45945 non-null  int64 
 14  09시-10시 하차인원  45945 non-null  int64 
 15  10시-11시 승차인원  45945 non-null  int64 
 16  10시-11시 하차인원  45945 non-null  int64 
 17  11시-12시 승차인원  45945 non-null  int64 
 18  11시-12시 하차인원  45945 non-null  int64 
 19  12시-

## tidy data
* melt

In [7]:
df = raw.melt(id_vars=["사용월", "호선명", "지하철역"], var_name="시간승하차", value_name="인원수")
df.head()

,사용월,호선명,지하철역,시간승하차,인원수
0,202107,1호선,동대문,04시-05시 승차인원,665
1,202107,1호선,동묘앞,04시-05시 승차인원,92
2,202107,1호선,서울역,04시-05시 승차인원,654
3,202107,1호선,시청,04시-05시 승차인원,13
4,202107,1호선,신설동,04시-05시 승차인원,295


In [8]:
df["호선명"].unique()

array(['1호선', '2호선', '3호선', '4호선', '5호선', '6호선', '7호선', '8호선', '9호선',
       '9호선2~3단계', '경강선', '경부선', '경원선', '경의선', '경인선', '경춘선', '공항철도 1호선',
       '과천선', '분당선', '수인선', '안산선', '우이신설선', '일산선', '장항선', '중앙선', '9호선2단계'],
      dtype=object)

In [9]:
df["시간승하차"].unique()

array(['04시-05시 승차인원', '04시-05시 하차인원', '05시-06시 승차인원', '05시-06시 하차인원',
       '06시-07시 승차인원', '06시-07시 하차인원', '07시-08시 승차인원', '07시-08시 하차인원',
       '08시-09시 승차인원', '08시-09시 하차인원', '09시-10시 승차인원', '09시-10시 하차인원',
       '10시-11시 승차인원', '10시-11시 하차인원', '11시-12시 승차인원', '11시-12시 하차인원',
       '12시-13시 승차인원', '12시-13시 하차인원', '13시-14시 승차인원', '13시-14시 하차인원',
       '14시-15시 승차인원', '14시-15시 하차인원', '15시-16시 승차인원', '15시-16시 하차인원',
       '16시-17시 승차인원', '16시-17시 하차인원', '17시-18시 승차인원', '17시-18시 하차인원',
       '18시-19시 승차인원', '18시-19시 하차인원', '19시-20시 승차인원', '19시-20시 하차인원',
       '20시-21시 승차인원', '20시-21시 하차인원', '21시-22시 승차인원', '21시-22시 하차인원',
       '22시-23시 승차인원', '22시-23시 하차인원', '23시-24시 승차인원', '23시-24시 하차인원',
       '00시-01시 승차인원', '00시-01시 하차인원', '01시-02시 승차인원', '01시-02시 하차인원',
       '02시-03시 승차인원', '02시-03시 하차인원', '03시-04시 승차인원', '03시-04시 하차인원',
       '작업일자'], dtype=object)

In [10]:
print(df.shape)
df = df[df["시간승하차"] != "작업일자"].copy()
print(df.shape)

(2251305, 5)
(2205360, 5)


In [11]:
df.describe()

,사용월,인원수
count,2.205360e+06,2.205360e+06
mean,2.017933e+05,1.490138e+04
std,1.902518e+02,2.396117e+04
min,2.015010e+05,0.000000e+00
25%,2.016090e+05,1.086000e+03
50%,2.018050e+05,7.182000e+03
75%,2.019120e+05,1.872200e+04
max,2.021070e+05,4.654060e+05


In [12]:
df.describe(include="object")

,호선명,지하철역,시간승하차
count,2205360,2205360,2205360
unique,26,579,48
top,5호선,서울역,04시-05시 승차인원
freq,195264,20832,45945


## 파생변수 만들기
### 문자열 나누기

In [13]:
df["시간"] = df["시간승하차"].str.split(expand=True)[0]
df["승하차"] = df["시간승하차"].str.split(expand=True)[1]
df.head(1)

,사용월,호선명,지하철역,시간승하차,인원수,시간,승하차
0,202107,1호선,동대문,04시-05시 승차인원,665,04시-05시,승차인원


### 출퇴근 시간 변수 만들기

In [14]:
df["시간대"] = df["시간"].str[:2].astype(int)
df["시간대"]

0          4
1          4
2          4
3          4
4          4
          ..
2205355    3
2205356    3
2205357    3
2205358    3
2205359    3
Name: 시간대, Length: 2205360, dtype: int64

In [15]:
df.loc[(df["시간대"] >= 7) & (df["시간대"] <= 10), "출퇴근"] = "출근"
df.loc[(df["시간대"] >= 17) & (df["시간대"] <= 21), "출퇴근"] = "퇴근"
df.head()

,사용월,호선명,지하철역,시간승하차,인원수,시간,승하차,시간대,출퇴근
0,202107,1호선,동대문,04시-05시 승차인원,665,04시-05시,승차인원,4,NaN
1,202107,1호선,동묘앞,04시-05시 승차인원,92,04시-05시,승차인원,4,NaN
2,202107,1호선,서울역,04시-05시 승차인원,654,04시-05시,승차인원,4,NaN
3,202107,1호선,시청,04시-05시 승차인원,13,04시-05시,승차인원,4,NaN
4,202107,1호선,신설동,04시-05시 승차인원,295,04시-05시,승차인원,4,NaN


### 문자열 대체
* 승차인원 => 승차
* 하차인원 => 하차
* "인원" 텍스트 제거

In [16]:
df["승하차"] = df["승하차"].str.replace("인원", "")
df.head(1)

,사용월,호선명,지하철역,시간승하차,인원수,시간,승하차,시간대,출퇴근
0,202107,1호선,동대문,04시-05시 승차인원,665,04시-05시,승차,4,NaN


### 문자열 슬라이싱
* "연도월" 형태의 데이터를 특정 인덱스를 기준으로 슬라이싱

In [17]:
df["연도"] = df["사용월"].astype(str).str[:4]
df["월"] = df["사용월"].astype(str).str[4:]
df.head(1)

,사용월,호선명,지하철역,시간승하차,인원수,시간,승하차,시간대,출퇴근,연도,월
0,202107,1호선,동대문,04시-05시 승차인원,665,04시-05시,승차,4,NaN,2021,07


In [ ]:
sns.barplot(data=df.sort_values("연도"), x="연도", y="인원수", ci=None, estimator=np.sum)
plt.title("연도별 합계 인원")

## 코로나 이전 이후 비교
### 특정 연도 추출

In [ ]:
df_192021 = df[df["연도"].isin(["2019", "2020", "2021"])]
df.shape, df_192021.shape

### 연도별 합계 인원

In [ ]:
sns.barplot(data=df_192021, x="연도", y="인원수", ci=None, estimator=np.sum)
plt.title("연도별 합계 인원")

### 호선별 합계 인원

In [ ]:
plt.figure(figsize=(15, 6))
sns.barplot(data=df_192021, x="인원수", y="호선명", ci=None, estimator=np.sum, hue="연도")
plt.title("호선별 합계 인원")

In [ ]:
df_line_year = df.groupby(["호선명", "연도"])["인원수"].sum().unstack()
df_line_year.style.background_gradient().format("{:,.0f}")

## 월별 승하차 인원

In [ ]:
plt.figure(figsize=(15, 4))
plt.xticks(rotation=60)
sns.pointplot(data=df_192021, x="사용월", y="인원수", hue="승하차", ci=None)

In [ ]:
plt.figure(figsize=(15, 4))
plt.xticks(rotation=60)
sns.pointplot(data=df_192021, x="사용월", y="인원수", hue="승하차", ci=None, estimator=np.sum)

## 호선별 인원

In [ ]:
plt.figure(figsize=(15, 6))
sns.barplot(data=df, x="인원수", y="호선명", ci=None)
plt.title("호선별 평균 인원")

In [ ]:
plt.figure(figsize=(15, 6))
sns.barplot(data=df, x="인원수", y="호선명", ci=None, estimator=sum)
plt.title("호선별 합계 인원")

In [ ]:
plt.figure(figsize=(15, 6))
sns.barplot(data=df, x="인원수", y="호선명", ci=None, estimator=sum, hue="승하차")
plt.title("호선별 합계 인원")

In [ ]:
plt.figure(figsize=(12, 4))
plt.xticks(rotation=80)
sns.pointplot(data=df, x="시간", y="인원수", ci=None, hue="승하차")

In [ ]:
plt.figure(figsize=(12, 4))
plt.xticks(rotation=80)
sns.pointplot(data=df_192021, x="시간", y="인원수", ci=None, hue="연도", estimator=np.sum)

In [ ]:
sns.catplot(data=df_192021, x="시간", y="인원수", kind="point",
            ci=None, hue="승하차", col="호선명", col_wrap=4)

## 승하차 인원

In [ ]:
df_192021[df_192021["승하차"] == "하차"].sort_values(by=["인원수", "지하철역"], ascending=False).head(20)

In [ ]:
df_192021[df_192021["승하차"] == "승차"].sort_values(by=["인원수", "지하철역"], ascending=False).head(20)

## 출퇴근 시간 승하차 빈도수

In [ ]:
df_station = df_192021.groupby(by=["지하철역", "승하차", "시간", "출퇴근"])["인원수"].sum().reset_index()
df_station

In [ ]:
df_station[df_station["승하차"] == "하차"].sort_values("인원수", ascending=False).head(20)

In [ ]:
df_station[
    (df_station["승하차"] == "승차") & (df_station["출퇴근"] == "출근")
].sort_values("인원수", ascending=False).head(20)

In [ ]:
df_station[
    (df_station["승하차"] == "하차") & (df_station["출퇴근"] == "출근")
].sort_values("인원수", ascending=False).head(20)

In [ ]:
df_station[
    (df_station["승하차"] == "승차") & (df_station["출퇴근"] == "퇴근")
].sort_values("인원수", ascending=False).head(20)

In [ ]:
df_station[
    (df_station["승하차"] == "하차") & (df_station["출퇴근"] == "퇴근")
].sort_values("인원수", ascending=False).head(20)